In [1]:
!uv pip install streamlit opencv-python scipy pydub langchain_groq gtts TTS pyngrok ffmpeg-python SpeechRecognition gradio

Streaming output truncated to the last 5000 lines.
      Built gruut==2.2.3
      Built docopt==0.6.2
      Built bnnumerizer==0.0.2
⠸ Preparing packages... (50/65)
numpy      ------------------------------ 17.27 MiB/17.41 MiB
nvidia-nvjitlink-cu12 ------------------------------ 17.51 MiB/20.09 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 17.73 MiB/23.50 MiB
speechrecognition ------------------------------ 17.58 MiB/31.33 MiB
gradio     ------------------------------ 10.38 MiB/44.76 MiB
nvidia-curand-cu12 ------------------------------ 17.44 MiB/53.70 MiB
sudachidict-core ------------------------------ 17.84 MiB/68.74 MiB
nvidia-cusolver-cu12 ------------------------------ 17.51 MiB/122.01 MiB
nvidia-cusparse-cu12 ------------------------------ 17.78 MiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 17.51 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 17.35 MiB/346.60 MiB
   Building gruut-lang-es==2.0.1
   Building gruut-lang-de==2.0.1
    

In [5]:
%%writefile app.py
import os
import io
import base64
import tempfile
import concurrent.futures
import streamlit as st
import speech_recognition as sr
from gtts import gTTS
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from TTS.api import TTS
import soundfile as sf

# ----------------------------------------------------------------------------------------------------------------------
#----------Voice Cloning Setup
#--------------Writing the reference audio to a temporary file.
#------------------Generating TTS output with the cloned voice using tom_cruise voice you can use any voice.
# -----------------------------------------------------------------------------------------------------------------------

@st.cache_resource
def load_voice_cloning_model():
    """Load the voice cloning model once and cache it."""
    return TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=True)

@st.cache_resource
def load_reference_voice():
    """Load the reference voice audio from a file in the environment."""
    reference_voice_path = "/content/tom cruise.mp3"
    with open(reference_voice_path, "rb") as f:
        return f.read()

def generate_cloned_audio(text, audio_bytes, tts_model):
    """Generate audio using the voice cloning model and in-memory reference audio."""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as ref_fp:
        audio_data = io.BytesIO(audio_bytes)
        data, samplerate = sf.read(audio_data)
        sf.write(ref_fp.name, data, samplerate)

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as out_fp:
        tts_model.tts_to_file(
            text=text,
            speaker_wav=ref_fp.name,
            language="en",
            file_path=out_fp.name
        )
        generated_audio = open(out_fp.name, "rb").read()
    os.unlink(ref_fp.name)
    os.unlink(out_fp.name)
    return generated_audio

# -----------------------------------------------------------------------------------------
# Helper Functions
# -----------------------------------------------------------------------------------------
def transcribe_audio(audio_bytes):
    """Transcribe audio using SpeechRecognition."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(io.BytesIO(audio_bytes)) as source:
        audio_data_sr = recognizer.record(source)
    return recognizer.recognize_google(audio_data_sr)

def process_llm_query(recognized_text, api_key):
    """Process the LLM query using ChatGroq."""
    prompt_template = [
        ('system', 'You are a helpful assistant, your name is Assistant, developed by Ayush Kumar'),
        ('user', '{input}')
    ]
    prompt = ChatPromptTemplate.from_messages(prompt_template)
    llm = ChatGroq(api_key=api_key , model="llama3-8b-8192")
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({'input': recognized_text})

def generate_tts(response_text, reference_audio=None):
    """
    Generate TTS audio using voice cloning if a reference voice is provided,
    otherwise fallback to a default gTTS voice.
    """
    try:
        if reference_audio:
            tts_model = load_voice_cloning_model()
            return generate_cloned_audio(response_text, reference_audio, tts_model)
        else:
            tts = gTTS(text=response_text, lang='en')
            mp3_fp = io.BytesIO()
            tts.write_to_fp(mp3_fp)
            return mp3_fp.getvalue()
    except Exception as e:
        st.error(f"Error in TTS generation: {e}")
        return None

# ---------------------------------------------------------------------------------------------------------
#------Main Streamlit App
#---------Loading the reference voice.
#--------------- Recording and Process Query Voice
#--------------------Using the recorded audio solely for transcription (query text extraction)
#------------------------Transcribing the query
#------------------------------Processing query if wake word detected and if it’s a new query
#--------------------------------- Processing query if wake word detected and if it’s a new query
#----------------------------------------Converting audio to base64 string for embedding in HTML
# -----------------------------------------------------------------------------------------------------------
st.title("AI Assistant")
st.write("🎤  Record your voice query (say 'assistant' to trigger processing).🎤 ")


if "reference_audio" not in st.session_state:
    st.session_state.reference_audio = load_reference_voice()

if "recognized_text" not in st.session_state:
    st.session_state.recognized_text = ""
if "last_query" not in st.session_state:
    st.session_state.last_query = ""
if "response_text" not in st.session_state:
    st.session_state.response_text = ""
if "cloned_audio" not in st.session_state:
    st.session_state.cloned_audio = None

st.header("Record Your Query")
audio_file = st.audio_input("say 'assistant' to trigger processing")
if audio_file is not None:
    audio_bytes = audio_file.read()
    st.audio(audio_bytes, format="audio/wav")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_transcribe = executor.submit(transcribe_audio, audio_bytes)
        try:
            recognized_text = future_transcribe.result(timeout=20)
            st.write("**You said:**", recognized_text)
            st.session_state.recognized_text = recognized_text
        except Exception as e:
            st.error(f"Error transcribing audio: {e}")

    if ("assistant" in recognized_text.lower() and recognized_text != st.session_state.last_query):
        st.session_state.last_query = recognized_text
        st.write("🔔 Processing query...")

        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_llm = executor.submit(
                process_llm_query,
                recognized_text,
                "your_api"
            )
            try:
                response_text = future_llm.result(timeout=60)
                st.session_state.response_text = response_text
                st.write("**Response:**", response_text)

                with st.spinner("Generating cloned voice response..."):
                    cloned_audio = generate_tts(response_text, st.session_state.reference_audio)
                    if cloned_audio:
                        st.session_state.cloned_audio = cloned_audio
                        st.audio(cloned_audio, format="audio/wav" , autoplay=True)
            except Exception as e:
                st.error(f"Error processing LLM query: {e}")
else:
    st.info("Awaiting voice input.")


Overwriting app.py


In [6]:
from pyngrok import ngrok
import time

ngrok.set_auth_token("your_api")
public_url = ngrok.connect(8501).public_url
print("Access the UI at:", public_url)

!streamlit run app.py &
time.sleep(5)


Access the UI at: https://33be-34-87-132-218.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.132.218:8501



2025-04-25 06:33:24.459 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
                      ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 210, in <lambda>
    lambda m: list(m.__path__._path),
                   ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_classes.py", line 13, in __getattr__
    proxy = torch._C._get_custom_class_python_wrapper(self.name, attr)
            ^^^^^

KeyboardInterrupt: 